## Homework

In this homework, we'll deploy the ride duration model in batch mode. Like in homework 1, we'll use the Yellow Taxi Trip Records dataset. 

You'll find the starter code in the [homework](homework) directory.


## Q1. Notebook

We'll start with the same notebook we ended up with in homework 1.
We cleaned it a little bit and kept only the scoring part. You can find the initial notebook [here](homework/starter.ipynb).

Run this notebook for the March 2023 data.

What's the standard deviation of the predicted duration for this dataset?

* 1.24
* 6.24
* 12.28
* 18.28

In [2]:
# 1. conda create -n "04-dep-2024" python=3.10.13 ipython
# 2. conda activate 04-dep-2024 
# 3. pip install scikit-learn==1.5.0
# 4. pip install pandas
# 5. pip install pyarrow
# 6. Load March 2023 data 'df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')'
# 7. y_pred.std()

# -> 6.247488852238703

## Q2. Preparing the output

Like in the course videos, we want to prepare the dataframe with the output. 

First, let's create an artificial `ride_id` column:

```python
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
```

Next, write the ride id and the predictions to a dataframe with results. 

Save it as parquet:

```python
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)
```

What's the size of the output file?

* 36M
* 46M
* 56M
* 66M


__Note:__ Make sure you use the snippet above for saving the file. It should contain only these two columns. For this question, don't change the
dtypes of the columns and use `pyarrow`, not `fastparquet`. 

In [ ]:
# 1. Create output folder
# 2. CODE:
# year = 2023
# month = 3
# df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
# df_result = pd.DataFrame({'ride_id': df['ride_id'], 'predictions': y_pred})

# df_result.to_parquet(
#     'output/predictions_2022-02.parquet',
#     engine='pyarrow',
#     compression=None,
#     index=False
# )

# !du -h ./output/predictions_2022-02.parquet

# -> 66M

## Q3. Creating the scoring script

Now let's turn the notebook into a script. 

Which command you need to execute for that?

In [ ]:
# 1. pip install nbconvert
# 2. cd 2024_mlops_homework/04-deployment/2024/homework
# 3. jupyter nbconvert --to script starter.ipynb

## Q4. Virtual environment

Now let's put everything into a virtual environment. We'll use pipenv for that.

Install all the required libraries. Pay attention to the Scikit-Learn version: it should be the same as in the starter
notebook.

After installing the libraries, pipenv creates two files: `Pipfile`
and `Pipfile.lock`. The `Pipfile.lock` file keeps the hashes of the
dependencies we use for the virtual env.

What's the first hash for the Scikit-Learn dependency?

In [3]:
# 1. Oops, created in Q1
# 2. Leave conda env
# 3. pipenv install scikit-learn==1.5.0 pandas pyarrow
# -> sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c
# 4. pipenv shell

## Q5. Parametrize the script

Let's now make the script configurable via CLI. We'll create two 
parameters: year and month.

Run the script for April 2023. 

What's the mean predicted duration? 

* 7.29
* 14.29
* 21.29
* 28.29

Hint: just add a print statement to your script.

In [ ]:
# 1. Add 'import sys'
# 2. Create def run()
# 3. Add:
        # year = int(sys.argv[1])
        # month = int(sys.argv[2])
        # input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
        # output_file = f'output/predictions_{year:04d}-{month:02d}.parquet'

        # df = read_data(input_file)
        # print(f'The mean predicted duration is {y_pred.mean()}.')
# 4. Add:
# if __name__ == '__main__':
#      run()
# 4. Run python starter.py 2023 4

# -> The mean predicted duration is 14.292282936862449.

## Q6. Docker container 

Finally, we'll package the script in the docker container. 
For that, you'll need to use a base image that we prepared. 

This is what the content of this image is:
```
FROM python:3.10.13-slim

WORKDIR /app
COPY [ "model2.bin", "model.bin" ]
```

Note: you don't need to run it. We have already done it.

It is pushed it to [`agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim`](https://hub.docker.com/layers/agrigorev/zoomcamp-model/mlops-2024-3.10.13-slim/images/sha256-f54535b73a8c3ef91967d5588de57d4e251b22addcbbfb6e71304a91c1c7027f?context=repo),
which you need to use as your base image.

That is, your Dockerfile should start with:

```docker
FROM agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim

# do stuff here
```

This image already has a pickle file with a dictionary vectorizer
and a model. You will need to use them.

Important: don't copy the model to the docker image. You will need
to use the pickle file already in the image. 

Now run the script with docker. What's the mean predicted duration
for May 2023? 

* 0.19
* 7.24
* 14.24
* 21.19

In [ ]:
# 1. Create Dockerfile:
# # Dockerfile
# FROM agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim

# RUN pip install -U pip
# RUN pip install pipenv

# WORKDIR /app
# COPY ["Pipfile", "Pipfile.lock", "starter.py", "./"]

# RUN pipenv install --system --deploy

# RUN mkdir -p /app/output
# #COPY ["model.bin", "./"]

# # Set the entrypoint to run the prediction script
# ENTRYPOINT ["python", "starter.py"]

# 2. docker build -t ride-duration-predictor .

# 3. docker run --rm ride-duration-predictor 2023 3